## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [41]:
# Importar bibliotecas necessárias
import os
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import select, func
from google.colab import files  # Para upload no Google Colab

# PASSO 1: Solicitar ao usuário o upload do arquivo 'chinook.db'
print("Por favor, faça o upload do arquivo 'chinook.db'.")
uploaded = files.upload()  # Upload no Google Colab

# Verificar se algum arquivo foi carregado
if not uploaded:
    raise FileNotFoundError(
        "Nenhum arquivo foi carregado. Por favor, baixe e extraia o arquivo 'chinook.zip' "
        "do link: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip e faça o upload novamente."
    )

# Salvar o arquivo com o nome 'chinook.db'
with open('chinook.db', 'wb') as f:
    for filename, file_content in uploaded.items():
        f.write(file_content)

# Verificar se o arquivo foi salvo corretamente
if not os.path.exists('chinook.db'):
    raise FileNotFoundError("Erro ao salvar o arquivo 'chinook.db'. Tente novamente.")

print("\nArquivo 'chinook.db' carregado com sucesso!")

Por favor, faça o upload do arquivo 'chinook.db'.


Saving chinook.db to chinook (4).db

Arquivo 'chinook.db' carregado com sucesso!


In [42]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

# PASSO 1: Conectar ao banco de dados SQLite
engine = create_engine('sqlite:///chinook.db')  # Certifique-se de que 'chinook.db' está carregado
Session = sessionmaker(bind=engine)
session = Session()

# PASSO 2: Configurar o mapeamento ORM automaticamente
Base = automap_base()
Base.prepare(autoload_with=engine)  # Usar autoload_with em vez de reflect=True

# Acessar a classe mapeada para a tabela 'albums'
Album = Base.classes.albums

# PASSO 3: Consultar o primeiro álbum
first_album = session.query(Album).first()

# Exibir o ID e o título do primeiro álbum
if first_album:
    print(f"AlbumId: {first_album.AlbumId}, Title: {first_album.Title}")
else:
    print("Nenhum álbum encontrado na tabela.")

AlbumId: 1, Title: For Those About To Rock We Salute You


#### Q3.
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [49]:
# Importar bibliotecas necessárias
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

# PASSO 1: Conectar ao banco de dados SQLite
engine = create_engine('sqlite:///chinook.db')  # Certifique-se de que 'chinook.db' está carregado
Session = sessionmaker(bind=engine)
session = Session()

# PASSO 2: Carregar as tabelas do banco de dados
metadata = MetaData()
metadata.reflect(bind=engine)  # Associar a engine ao refletir as tabelas

# Acessar as tabelas principais
tracks_table = metadata.tables['tracks']
invoice_items_table = metadata.tables['invoice_items']

# PASSO 3: Imprimir os três primeiros registros da tabela tracks
query = (
    session.query(tracks_table.c.Name, func.sum(invoice_items_table.c.Quantity).label('total_vendido'))
    .join(invoice_items_table, tracks_table.c.TrackId == invoice_items_table.c.TrackId)
    .group_by(tracks_table.c.Name)
    .order_by(func.sum(invoice_items_table.c.Quantity).desc())
    .limit(10)
)

print("\nAs 10 faixas mais vendidas:")
for result in query:  # Corrigindo a sintaxe aqui
    print(f"- Faixa: {result.Name}, Total Vendido: {result.total_vendido}")


As 10 faixas mais vendidas:
- Faixa: The Trooper, Total Vendido: 5
- Faixa: Untitled, Total Vendido: 4
- Faixa: The Number Of The Beast, Total Vendido: 4
- Faixa: Sure Know Something, Total Vendido: 4
- Faixa: Hallowed Be Thy Name, Total Vendido: 4
- Faixa: Eruption, Total Vendido: 4
- Faixa: Where Eagles Dare, Total Vendido: 3
- Faixa: Welcome Home (Sanitarium), Total Vendido: 3
- Faixa: Sweetest Thing, Total Vendido: 3
- Faixa: Surrender, Total Vendido: 3


In [51]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
# Importar bibliotecas necessárias
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# PASSO 1: Conectar ao banco de dados SQLite
engine = create_engine('sqlite:///chinook.db')  # Certifique-se de que 'chinook.db' está carregado
Session = sessionmaker(bind=engine)
session = Session()

# PASSO 2: Carregar as tabelas do banco de dados
metadata = MetaData()
metadata.reflect(bind=engine)  # Associar a engine ao refletir as tabelas

# Acessar as tabelas principais
tracks_table = metadata.tables['tracks']
albums_table = metadata.tables['albums']

# PASSO 3: Imprimir o nome da faixa e o título do álbum das primeiras 20 faixas
query = (
    session.query(tracks_table.c.Name, albums_table.c.Title)
    .join(albums_table, tracks_table.c.AlbumId == albums_table.c.AlbumId)
    .limit(20)
)

print("\nNome da faixa e título do álbum das primeiras 20 faixas:")
for result in query:
    print(f"- Faixa: {result.Name}, Álbum: {result.Title}")


Nome da faixa e título do álbum das primeiras 20 faixas:
- Faixa: For Those About To Rock (We Salute You), Álbum: For Those About To Rock We Salute You
- Faixa: Put The Finger On You, Álbum: For Those About To Rock We Salute You
- Faixa: Let's Get It Up, Álbum: For Those About To Rock We Salute You
- Faixa: Inject The Venom, Álbum: For Those About To Rock We Salute You
- Faixa: Snowballed, Álbum: For Those About To Rock We Salute You
- Faixa: Evil Walks, Álbum: For Those About To Rock We Salute You
- Faixa: C.O.D., Álbum: For Those About To Rock We Salute You
- Faixa: Breaking The Rules, Álbum: For Those About To Rock We Salute You
- Faixa: Night Of The Long Knives, Álbum: For Those About To Rock We Salute You
- Faixa: Spellbound, Álbum: For Those About To Rock We Salute You
- Faixa: Balls to the Wall, Álbum: Balls to the Wall
- Faixa: Fast As a Shark, Álbum: Restless and Wild
- Faixa: Restless and Wild, Álbum: Restless and Wild
- Faixa: Princess of the Dawn, Álbum: Restless and Wild


In [54]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.

# Importar bibliotecas necessárias
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# PASSO 1: Conectar ao banco de dados SQLite
engine = create_engine('sqlite:///chinook.db')  # Certifique-se de que 'chinook.db' está carregado
Session = sessionmaker(bind=engine)
session = Session()

# PASSO 2: Carregar as tabelas do banco de dados
metadata = MetaData()
metadata.reflect(bind=engine)  # Associar a engine ao refletir as tabelas

# Acessar as tabelas principais
tracks_table = metadata.tables['tracks']
invoice_items_table = metadata.tables['invoice_items']

# PASSO 3: Imprimir as 10 primeiras vendas de faixas
query = (
    session.query(invoice_items_table.c.InvoiceId, invoice_items_table.c.TrackId, invoice_items_table.c.Quantity, invoice_items_table.c.UnitPrice)
    .limit(10)
)

print("\n10 primeiras vendas de faixas:")
for item in query:
    print(f"- InvoiceId: {item.InvoiceId}, TrackId: {item.TrackId}, Quantidade: {item.Quantity}, Preço: {item.UnitPrice}")

# PASSO 4: Para essas 10 primeiras vendas, imprimir os nomes das faixas vendidas e a quantidade vendida
query = (
    session.query(tracks_table.c.Name, invoice_items_table.c.Quantity)
    .join(invoice_items_table, tracks_table.c.TrackId == invoice_items_table.c.TrackId)
    .limit(10)
)

print("\nNomes das faixas vendidas e quantidade vendida:")
for track_name, quantity in query:
    print(f"- Faixa: {track_name}, Quantidade: {quantity}")


10 primeiras vendas de faixas:
- InvoiceId: 1, TrackId: 2, Quantidade: 1, Preço: 0.99
- InvoiceId: 1, TrackId: 4, Quantidade: 1, Preço: 0.99
- InvoiceId: 2, TrackId: 6, Quantidade: 1, Preço: 0.99
- InvoiceId: 2, TrackId: 8, Quantidade: 1, Preço: 0.99
- InvoiceId: 2, TrackId: 10, Quantidade: 1, Preço: 0.99
- InvoiceId: 2, TrackId: 12, Quantidade: 1, Preço: 0.99
- InvoiceId: 3, TrackId: 16, Quantidade: 1, Preço: 0.99
- InvoiceId: 3, TrackId: 20, Quantidade: 1, Preço: 0.99
- InvoiceId: 3, TrackId: 24, Quantidade: 1, Preço: 0.99
- InvoiceId: 3, TrackId: 28, Quantidade: 1, Preço: 0.99

Nomes das faixas vendidas e quantidade vendida:
- Faixa: Balls to the Wall, Quantidade: 1
- Faixa: Restless and Wild, Quantidade: 1
- Faixa: Put The Finger On You, Quantidade: 1
- Faixa: Inject The Venom, Quantidade: 1
- Faixa: Evil Walks, Quantidade: 1
- Faixa: Breaking The Rules, Quantidade: 1
- Faixa: Dog Eat Dog, Quantidade: 1
- Faixa: Overdose, Quantidade: 1
- Faixa: Love In An Elevator, Quantidade: 1
-

In [55]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
query = (
    session.query(tracks_table.c.Name, func.sum(invoice_items_table.c.Quantity).label('total_vendido'))
    .join(tracks_table, invoice_items_table.c.TrackId == tracks_table.c.TrackId)
    .group_by(tracks_table.c.Name)
    .order_by(func.sum(invoice_items_table.c.Quantity).desc())
    .limit(10)
)

print("\n10 faixas mais vendidas:")
for track_name, total_sold in query:
    print(f"- Faixa: {track_name}, Total Vendido: {total_sold}")




10 faixas mais vendidas:
- Faixa: The Trooper, Total Vendido: 5
- Faixa: Untitled, Total Vendido: 4
- Faixa: The Number Of The Beast, Total Vendido: 4
- Faixa: Sure Know Something, Total Vendido: 4
- Faixa: Hallowed Be Thy Name, Total Vendido: 4
- Faixa: Eruption, Total Vendido: 4
- Faixa: Where Eagles Dare, Total Vendido: 3
- Faixa: Welcome Home (Sanitarium), Total Vendido: 3
- Faixa: Sweetest Thing, Total Vendido: 3
- Faixa: Surrender, Total Vendido: 3


In [59]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
# Importar bibliotecas necessárias
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

# PASSO 1: Conectar ao banco de dados SQLite
engine = create_engine('sqlite:///chinook.db')  # Certifique-se de que 'chinook.db' está carregado
Session = sessionmaker(bind=engine)
session = Session()

# PASSO 2: Carregar as tabelas do banco de dados
metadata = MetaData()
metadata.reflect(bind=engine)  # Associar a engine ao refletir as tabelas

# Acessar as tabelas principais
invoice_items_table = metadata.tables['invoice_items']
tracks_table = metadata.tables['tracks']
albums_table = metadata.tables['albums']
artists_table = metadata.tables['artists']

# PASSO 3: Consulta para encontrar os 10 artistas que mais venderam
query = (
    session.query(artists_table.c.Name, func.sum(invoice_items_table.c.Quantity).label('total_vendido'))
    .join(tracks_table, invoice_items_table.c.TrackId == tracks_table.c.TrackId)
    .join(albums_table, tracks_table.c.AlbumId == albums_table.c.AlbumId)
    .join(artists_table, albums_table.c.ArtistId == artists_table.c.ArtistId)
    .group_by(artists_table.c.Name)
    .order_by(func.sum(invoice_items_table.c.Quantity).desc())
    .limit(10)
)

print("\n10 artistas que mais venderam:")
for artist_name, total_sold in query:
    print(f"- Artista: {artist_name}, Total Vendido: {total_sold}")


10 artistas que mais venderam:
- Artista: Iron Maiden, Total Vendido: 140
- Artista: U2, Total Vendido: 107
- Artista: Metallica, Total Vendido: 91
- Artista: Led Zeppelin, Total Vendido: 87
- Artista: Os Paralamas Do Sucesso, Total Vendido: 45
- Artista: Deep Purple, Total Vendido: 44
- Artista: Faith No More, Total Vendido: 42
- Artista: Lost, Total Vendido: 41
- Artista: Eric Clapton, Total Vendido: 40
- Artista: R.E.M., Total Vendido: 39
